В данной задаче нам представлены данные в которых ключевое поле — это то, просрочит ли клиент микро-финансовой организации выплату более чем на 60 дней или нет. 

Решить задачу с использованием torch

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from datetime import datetime

file_path = 'MFOcredit.csv'

In [2]:
df = pd.read_csv(file_path, encoding='cp1251', sep=';')

In [3]:
df.head()

,id,date_start,date_end,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus
0,1,03-Jan-2013,12-Jan-2013,Мужской,44,Нет,Собственное,Гражданский брак/женат/замужем,Нет,Официальное,Новосибирская область,Нет,Да,Нет
1,2,03-Jan-2013,17-Jan-2013,Мужской,21,Пропуск поля,Живут с родителями,Холост,Нет,Официальное,Кемеровская область юг,Да,Нет,Нет
2,3,03-Jan-2013,17-Jan-2013,Мужской,25,Пропуск поля,Собственное,Холост,Да,Официальное,Кемеровская область север,Пропуск поля,Нет,Нет
3,4,03-Jan-2013,17-Jan-2013,Женский,47,Пропуск поля,Собственное,Гражданский брак/женат/замужем,Да,Официальное,Кемеровская область север,Нет,Нет,Нет
4,5,03-Jan-2013,17-Jan-2013,Мужской,22,Нет,Арендуемое,Гражданский брак/женат/замужем,Нет,Официальное,Кемеровская область север,Да,Да,Нет


In [4]:
df = df.drop(['id'], axis = 1)

In [5]:
df.head()

,date_start,date_end,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus
0,03-Jan-2013,12-Jan-2013,Мужской,44,Нет,Собственное,Гражданский брак/женат/замужем,Нет,Официальное,Новосибирская область,Нет,Да,Нет
1,03-Jan-2013,17-Jan-2013,Мужской,21,Пропуск поля,Живут с родителями,Холост,Нет,Официальное,Кемеровская область юг,Да,Нет,Нет
2,03-Jan-2013,17-Jan-2013,Мужской,25,Пропуск поля,Собственное,Холост,Да,Официальное,Кемеровская область север,Пропуск поля,Нет,Нет
3,03-Jan-2013,17-Jan-2013,Женский,47,Пропуск поля,Собственное,Гражданский брак/женат/замужем,Да,Официальное,Кемеровская область север,Нет,Нет,Нет
4,03-Jan-2013,17-Jan-2013,Мужской,22,Нет,Арендуемое,Гражданский брак/женат/замужем,Нет,Официальное,Кемеровская область север,Да,Да,Нет


In [6]:
# преобразовали наши столбцы времени в формат даты
df['date_start'] = pd.to_datetime(df['date_start'])
df['date_end'] = pd.to_datetime(df['date_end'])

Из наших полей date_start и date_end получим год, квартал, месяц и день. А так же посчитаем разницу в днях между датой начала и ожидаемой датой окончания.

In [7]:
df.insert(0, 'start_year', df['date_start'].dt.year)
df.insert(1, 'start_quart', df['date_start'].dt.quarter)
df.insert(2, 'start_month', (df['date_start']).dt.month)
df.insert(3, 'start_day', (df['date_start']).dt.day)

df.insert(4, 'end_year', df['date_end'].dt.year)
df.insert(5, 'end_quart', df['date_end'].dt.quarter)
df.insert(6, 'end_month', df['date_end'].dt.month)
df.insert(7, 'end_day', (df['date_start']).dt.day)

In [8]:
df.insert(8, 'date_diff', (df['date_end'] - df['date_start']).dt.days)

In [9]:
df = df.drop(['date_start', 'date_end'], axis = 1)

In [10]:
df.head()

,start_year,start_quart,start_month,start_day,end_year,end_quart,end_month,end_day,date_diff,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus
0,2013,1,1,3,2013,1,1,3,9,Мужской,44,Нет,Собственное,Гражданский брак/женат/замужем,Нет,Официальное,Новосибирская область,Нет,Да,Нет
1,2013,1,1,3,2013,1,1,3,14,Мужской,21,Пропуск поля,Живут с родителями,Холост,Нет,Официальное,Кемеровская область юг,Да,Нет,Нет
2,2013,1,1,3,2013,1,1,3,14,Мужской,25,Пропуск поля,Собственное,Холост,Да,Официальное,Кемеровская область север,Пропуск поля,Нет,Нет
3,2013,1,1,3,2013,1,1,3,14,Женский,47,Пропуск поля,Собственное,Гражданский брак/женат/замужем,Да,Официальное,Кемеровская область север,Нет,Нет,Нет
4,2013,1,1,3,2013,1,1,3,14,Мужской,22,Нет,Арендуемое,Гражданский брак/женат/замужем,Нет,Официальное,Кемеровская область север,Да,Да,Нет


Закодируем признаки с использованием OneHotEncoder

In [11]:
oh = OneHotEncoder()

In [12]:
coder1 = oh.fit_transform(df[['gender']])
coder2 = oh.fit_transform(df[['auto']])
coder3 = oh.fit_transform(df[['housing']])
coder4 = oh.fit_transform(df[['marstatus']])
coder5 = oh.fit_transform(df[['regclient']])
coder6 = oh.fit_transform(df[['jobtype']])
coder7 = oh.fit_transform(df[['region']])
coder8 = oh.fit_transform(df[['credits']])
coder9 = oh.fit_transform(df[['children']])

In [13]:
dff1 = pd.DataFrame(coder1.toarray(), index = df.index, columns = ['female','male'])
dff2 = pd.DataFrame(coder2.toarray(), index = df.index, columns = ['есть машины','нет машины', 'NA авто'])
dff3 = pd.DataFrame(coder3.toarray(), index = df.index, columns = ['Арендуемое', 'Долевая собственность', 'Живут с родителями', 'Муниципальное', 'Пропуск поля', 'Собственное'])
dff4 = pd.DataFrame(coder4.toarray(), index = df.index, columns = ['Вдова/вдовец', 'Гражданский брак/женат/замужем', 'NA СП', 'Разведен', 'Холост'])
dff5 = pd.DataFrame(coder5.toarray(), index = df.index, columns = ['Постоянный клиент','Не постоянный клиент'])
dff6 = pd.DataFrame(coder6.toarray(), index = df.index, columns = ['Нефициальное', 'Официальное', 'NA работа'])
dff7 = pd.DataFrame(coder7.toarray(), index = df.index, columns = ['Алтайский край','Кемеровская область север', 'Кемеровская область юг', 'Красноярский край', 'Новосибирская область'])
dff8 = pd.DataFrame(coder8.toarray(), index = df.index, columns = ['есть кредит','нет кредита', 'NA кредит'])
dff9 = pd.DataFrame(coder9.toarray(), index = df.index, columns = ['есть дети','нет детей', 'NA дети'])

In [14]:
df = pd.concat([df, dff1], axis = 1)
df = pd.concat([df, dff2], axis = 1)
df = pd.concat([df, dff3], axis = 1)
df = pd.concat([df, dff4], axis = 1)
df = pd.concat([df, dff5], axis = 1)
df = pd.concat([df, dff6], axis = 1)
df = pd.concat([df, dff7], axis = 1)
df = pd.concat([df, dff8], axis = 1)
df = pd.concat([df, dff9], axis = 1)

In [15]:
df.head()

,start_year,start_quart,start_month,start_day,end_year,end_quart,end_month,end_day,date_diff,gender,...,Кемеровская область север,Кемеровская область юг,Красноярский край,Новосибирская область,есть кредит,нет кредита,NA кредит,есть дети,нет детей,NA дети
0,2013,1,1,3,2013,1,1,3,9,Мужской,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,2013,1,1,3,2013,1,1,3,14,Мужской,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,2013,1,1,3,2013,1,1,3,14,Мужской,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,2013,1,1,3,2013,1,1,3,14,Женский,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,2013,1,1,3,2013,1,1,3,14,Мужской,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [16]:
df = df.drop(['gender', 'auto', 'housing', 'marstatus', 'regclient', 'jobtype', 'credits', 'children','region'], axis = 1)

In [17]:
df.head()

,start_year,start_quart,start_month,start_day,end_year,end_quart,end_month,end_day,date_diff,age,...,Кемеровская область север,Кемеровская область юг,Красноярский край,Новосибирская область,есть кредит,нет кредита,NA кредит,есть дети,нет детей,NA дети
0,2013,1,1,3,2013,1,1,3,9,44,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,2013,1,1,3,2013,1,1,3,14,21,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,2013,1,1,3,2013,1,1,3,14,25,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,2013,1,1,3,2013,1,1,3,14,47,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,2013,1,1,3,2013,1,1,3,14,22,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [18]:
df['delinq60plus'] = df['delinq60plus'].replace({'Да': '1', 'Нет': '0'})

In [19]:
df.head()

,start_year,start_quart,start_month,start_day,end_year,end_quart,end_month,end_day,date_diff,age,...,Кемеровская область север,Кемеровская область юг,Красноярский край,Новосибирская область,есть кредит,нет кредита,NA кредит,есть дети,нет детей,NA дети
0,2013,1,1,3,2013,1,1,3,9,44,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,2013,1,1,3,2013,1,1,3,14,21,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,2013,1,1,3,2013,1,1,3,14,25,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,2013,1,1,3,2013,1,1,3,14,47,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,2013,1,1,3,2013,1,1,3,14,22,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


Разделение данных

In [20]:
X = df.drop(['delinq60plus'], axis = 1)

In [21]:
X.head(5)

,start_year,start_quart,start_month,start_day,end_year,end_quart,end_month,end_day,date_diff,age,...,Кемеровская область север,Кемеровская область юг,Красноярский край,Новосибирская область,есть кредит,нет кредита,NA кредит,есть дети,нет детей,NA дети
0,2013,1,1,3,2013,1,1,3,9,44,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,2013,1,1,3,2013,1,1,3,14,21,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,2013,1,1,3,2013,1,1,3,14,25,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,2013,1,1,3,2013,1,1,3,14,47,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,2013,1,1,3,2013,1,1,3,14,22,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [22]:
Y = df['delinq60plus']

In [23]:
Y.head(5)

0    0
1    0
2    0
3    0
4    0
Name: delinq60plus, dtype: object

Разделим наши данные на обучающие и тестовые

In [24]:
X_train, X_test, Y_train, Y_test =  train_test_split(X, Y, test_size = 0.2, random_state = 42)

Преобразуйте данные DataFrame в массивы NumPy

In [25]:
X_train_np = X_train.to_numpy(dtype=np.float32)
Y_train_np = Y_train.to_numpy(dtype=np.float32)
X_test_np = X_test.to_numpy(dtype=np.float32)
Y_test_np = Y_test.to_numpy(dtype=np.float32)

Создадим TensorDataset для обучающего и тестового наборов

In [26]:
train_dataset = TensorDataset(torch.from_numpy(X_train_np), torch.from_numpy(Y_train_np))
test_dataset = TensorDataset(torch.from_numpy(X_test_np), torch.from_numpy(Y_test_np))

Определим размер батчей и входных данных

In [27]:
input_size = 42
batch_size = 64

Создадим DataLoader для обучающих и тестовых данных

In [28]:
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Определим нашу модель

In [29]:
class BinaryClassificationModel(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassificationModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

Напишем функцию для обучения модели

In [30]:
def train(model, optimizer, loss_fn,  train_loader, val_loader, epochs=10):

    for epoch in range(epochs):
        
        training_loss = 0.0
        val_loss = 0.0
        
        model.train()

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels.unsqueeze(1).float())
            loss.backward()
            optimizer.step()
            training_loss += loss.item()

        # Оценка модели на валидационном наборе данных
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = loss_fn(outputs, labels.unsqueeze(1).float())
                val_loss += loss.item()
                predicted = (outputs >= 0.5).float()
                total += labels.size(0)
                correct += (predicted == labels.unsqueeze(1).float()).sum().item()

        print(f"Epoch {epoch+1}/{epochs}, Training Loss: {training_loss / len(train_loader):.4f}, Validation Loss: {val_loss / len(val_loader):.4f}, Accuracy: {correct / total:.4f}")

In [31]:
model = BinaryClassificationModel(input_size)

In [32]:
param = model.parameters()

In [33]:
train(model, optim.Adam(param, lr=0.001), nn.BCELoss(), train_data_loader, test_data_loader, epochs=10)

Epoch 1/10, Training Loss: 0.6504, Validation Loss: 0.5769, Accuracy: 0.6631
Epoch 2/10, Training Loss: 0.5597, Validation Loss: 0.5193, Accuracy: 0.7325
Epoch 3/10, Training Loss: 0.5497, Validation Loss: 0.8301, Accuracy: 0.6216
Epoch 4/10, Training Loss: 0.5349, Validation Loss: 0.5201, Accuracy: 0.7278
Epoch 5/10, Training Loss: 0.5211, Validation Loss: 0.6358, Accuracy: 0.6350
Epoch 6/10, Training Loss: 0.5139, Validation Loss: 0.5330, Accuracy: 0.6898
Epoch 7/10, Training Loss: 0.5197, Validation Loss: 0.4988, Accuracy: 0.7413
Epoch 8/10, Training Loss: 0.4977, Validation Loss: 0.5537, Accuracy: 0.7264
Epoch 9/10, Training Loss: 0.5063, Validation Loss: 0.4967, Accuracy: 0.7416
Epoch 10/10, Training Loss: 0.4935, Validation Loss: 0.4825, Accuracy: 0.7494
